In [1]:
import os, csv
import pandas as pd
import nibabel as nib
import ipywidgets
import matplotlib.pyplot as plt
import matplotlib as mpl
import nilearn.plotting
from IPython.display import display, clear_output
from ipywidgets import widgets, interact
import numpy as np
import regtricks as rt
from nibabel import nifti1
from PIL import Image
from nibabel.viewers import OrthoSlicer3D

%matplotlib notebook

D:\Softwares\Anaconda3\lib\site-packages\nilearn\datasets\__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML
javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Show code", True: "Hide code"}
def toggle_code(state):
    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)
    display(HTML(output))
def button_action(value):
    state = value.new
    toggle_code(state)
    value.owner.description = button_descriptions[state]
state = False
toggle_code(state)
button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")
display(button)

ToggleButton(value=False, description='Show code')

In [3]:
def getMasked(mask, image):
    temp =np.zeros(image.shape)
    num = 0 
    for id,x in np.ndenumerate(temp):
        if mask[id[0]][id[1]][id[2]] > 0:
            temp[id[0]][id[1]][id[2]] = image[id[0]][id[1]][id[2]]
            num += 1
        else:
            temp[id[0]][id[1]][id[2]] = np.nan
    print("mask shape:", mask.shape)
    print("mask pixls:", num)

    new_image = nib.Nifti1Image(temp, affine=np.eye(4))
    return new_image

subjects_list = []
for i in range(1,12):
    subjects_list.append('sub-'+str(i))

# GM & WM Perfusion

In [4]:
PVC_options = [("No", 0),('Yes', 1)]
PVC_dropdown = ipywidgets.Dropdown(options=PVC_options,
                                value=0,
                                description="PVC:")

Subject_options = subjects_list
Subject_dropdown = ipywidgets.Dropdown(options=Subject_options,
                                value='sub-1',
                                description="Subject:")

calib_list = [('voxelwise(default)', 0), ("csf", 1), ("wm", 2)]
Calib_options = calib_list
Calib_dropdown = ipywidgets.Dropdown(options=Calib_options,
                                value=0,
                                description="Calibration method:")

def get_csv_by_index(subject_name,pvc_index, calib_index):
    subject_path = os.path.join(os.getcwd(), 'oxasl_display', subject_name)
    calib_method = ['voxelwise', 'refregion']
    native_space_dir = os.path.join(subject_path, "oxasl_results", 'output', 'native')
    native_space_pvc_dir = os.path.join(subject_path, "oxasl_results", 'output_pvcorr', 'native')
    calib_folder = ['calib_voxelwise', 'calib_refregion_csf','calib_refregion_wm']
    region_analysis_dir = [] 
    region_analysis_pvc_dir = [] 
    for calib_f in calib_folder:
        #print(os.path.join(native_space_dir, calib_f))
        region_analysis_dir.append(os.path.join(native_space_dir, calib_f))
        region_analysis_pvc_dir.append(os.path.join(native_space_pvc_dir, calib_f))
    

    perfusions_list = []
    pd_Cerebral_Ratio_list = []
    pd_GM_WM_Ratio_list = []
    pd_LICA_RICA_Ratio_list = []
    for idx in range(len(region_analysis_dir)):
        perfusion_df = pd.read_csv(os.path.join(region_analysis_dir[idx], 'roi_stats.csv'))
        perfusion_gm_df = pd.read_csv(os.path.join(region_analysis_pvc_dir[idx], 'roi_stats_gm.csv'))
        perfusion_wm_df = pd.read_csv(os.path.join(region_analysis_pvc_dir[idx], 'roi_stats_wm.csv'))
    
        nonPVC = pd.concat([perfusion_df.iloc[0,:].to_frame().T, perfusion_df.iloc[2,:].to_frame().T, perfusion_df.iloc[1,:].to_frame().T, perfusion_df.iloc[3,:].to_frame().T,perfusion_df.iloc[4,:].to_frame().T,perfusion_df.iloc[5,:].to_frame().T,perfusion_df.iloc[24,:].to_frame().T,perfusion_df.iloc[25,:].to_frame().T,perfusion_df.iloc[26,:].to_frame().T], axis=0).reset_index(drop=True)
        PVC = pd.concat([perfusion_gm_df.iloc[0,].to_frame().T, perfusion_gm_df.iloc[2,].to_frame().T,perfusion_wm_df.iloc[1,].to_frame().T,perfusion_wm_df.iloc[3,].to_frame().T,perfusion_wm_df.iloc[4,:].to_frame().T,perfusion_wm_df.iloc[5,:].to_frame().T,perfusion_gm_df.iloc[24,:].to_frame().T,perfusion_gm_df.iloc[25,:].to_frame().T,perfusion_gm_df.iloc[26,].to_frame().T], axis=0).reset_index(drop=True)
        perfusions = [nonPVC, PVC]
        for perf in perfusions:
            perf['SpCov'] = perf[['Std', 'Mean']].apply(lambda x:x['Std']*100/x['Mean'], axis = 1)
        perfusions_list.append(perfusions)
    
        LICA_RICA_Ratio_nonPVC = {'LICA':[perfusion_df.iloc[25,2]], 'RICA':[perfusion_df.iloc[24,2]], "L/R":[round(perfusion_df.iloc[25,2]/perfusion_df.iloc[24,2], 2)]}
        pd_LICA_RICA_Ratio_nonPVC = pd.DataFrame(data=LICA_RICA_Ratio_nonPVC)
        LICA_RICA_Ratio_PVC = {'LICA':[perfusion_gm_df.iloc[25,2]], 'RICA':[perfusion_gm_df.iloc[24,2]], "L/R":[round(perfusion_gm_df.iloc[25,2]/perfusion_gm_df.iloc[24,2], 2)]}
        pd_LICA_RICA_Ratio_PVC = pd.DataFrame(data=LICA_RICA_Ratio_PVC)


        pd_LICA_RICA_Ratio = [pd_LICA_RICA_Ratio_nonPVC, pd_LICA_RICA_Ratio_PVC]
        pd_LICA_RICA_Ratio_list.append(pd_LICA_RICA_Ratio)
        
        Cerebral_Ratio_nonPVC = {'Right_Cerebral':[perfusion_df.iloc[4,2]], 'Left_Cerebral':[perfusion_df.iloc[5,2]], "R/L":[round(perfusion_df.iloc[4,2]/perfusion_df.iloc[5,2], 2)]}
        pd_Cerebral_Ratio_Ratio_nonPVC = pd.DataFrame(data=Cerebral_Ratio_nonPVC)
        Cerebral_Ratio_Ratio_PVC = {'Right_Cerebral':[perfusion_wm_df.iloc[4,2]], 'Left_Cerebral':[perfusion_wm_df.iloc[5,2]], "R/L":[round(perfusion_wm_df.iloc[4,2]/perfusion_wm_df.iloc[5,2], 2)]}
        pd_Cerebral_Ratio_Ratio_PVC = pd.DataFrame(data=Cerebral_Ratio_Ratio_PVC)

        pd_Cerebral_Ratio = [pd_Cerebral_Ratio_Ratio_nonPVC, pd_Cerebral_Ratio_Ratio_PVC]
        pd_Cerebral_Ratio_list.append(pd_Cerebral_Ratio)
        
        GM_WM_Ratio_nonPVC = {'80%+GM':[perfusion_df.iloc[2,2]], '90%+WM':[perfusion_df.iloc[3,2]], "80%+GM/90%+WM":[round(perfusion_df.iloc[2,2]/perfusion_df.iloc[3,2], 2)]}
        pd_GM_WM_Ratio_Ratio_nonPVC = pd.DataFrame(data=GM_WM_Ratio_nonPVC)
        GM_WM_Ratio_PVC = {'80%+GM':[perfusion_gm_df.iloc[2,2]], '90%+WM':[perfusion_wm_df.iloc[3,2]], "80%+GM/90%+WM":[round(perfusion_gm_df.iloc[2,2]/perfusion_wm_df.iloc[3,2], 2)]}
        pd_GM_WM_Ratio_PVC = pd.DataFrame(data=GM_WM_Ratio_PVC)

        pd_GM_WM_Ratio = [pd_GM_WM_Ratio_Ratio_nonPVC, pd_GM_WM_Ratio_PVC]
        pd_GM_WM_Ratio_list.append(pd_GM_WM_Ratio)
    
    return [perfusions_list, pd_LICA_RICA_Ratio_list, pd_Cerebral_Ratio_list, pd_GM_WM_Ratio_list ]


def mean_perfusion_barchart(data):
    fig=plt.figure(figsize=(5,3))
    ax= fig.add_subplot(1,1,1)
    plt.xticks((0, 1, 2, 3, 4, 5, 6, 7, 8), ('10%+GM', '80%+GM', '10%+WM', '90%+WM', 'RC80%WM','LC80%WM', 'RICA', 'LICA', 'VBA'))
    plt.title("Histgram of perfusion territories")
    plt.xlabel('Territory')
    plt.ylabel('Perfusion')
    data.set_index(data.columns[0], inplace=True)
    #data = data.T
    plt.xticks(size=8)
    data['Mean'].transpose().plot(kind='bar')
    plt.show()
    
def show_table(subject_name, pvc_index, calib_index):
    subject_id = int(subject_name[4:])
    try:
        pd_list= get_csv_by_index(subject_name, pvc_index,calib_index)
        display(pd_list[0][calib_index][pvc_index])
        display(pd_list[1][calib_index][pvc_index])
        display(pd_list[2][calib_index][pvc_index])
        display(pd_list[3][calib_index][pvc_index])
        mean_perfusion_barchart(pd_list[0][calib_index][pvc_index])
    except Exception as e:
        print(e)
        print("Region_analysis file doesn't exist.")

_ = interact(show_table,subject_name=Subject_dropdown, pvc_index=PVC_dropdown, calib_index=Calib_dropdown)



interactive(children=(Dropdown(description='Subject:', options=('sub-1', 'sub-2', 'sub-3', 'sub-4', 'sub-5', '…

# Pre & post Distortion Correction prefusions

In [5]:
#new version
Subject_options = subjects_list
Subject_dropdown = ipywidgets.Dropdown(options=Subject_options,
                                value='sub-1',
                                description="Subject:")


slice_slider = ipywidgets.IntSlider(value=6,
                                    min=0,
                                    max=12,
                                    step=1,
                                    description='Slice:',)

pwi_list = [("not dc and mc", 0), ("not dc but mc",1), ("dc and mc",2)]
PWI_options = pwi_list
PWI_dropdown = ipywidgets.Dropdown(options=PWI_options,
                                value=0,
                                description="PWI:")

PWI_dropdown2 = ipywidgets.Dropdown(options=PWI_options,
                                value=0,
                                description="PWI2:")

def show_difference_map(subject_name, pwi_index, pwi_index2, slice_index=6):
    subject_id = int(subject_name[4:]) 
    subject_path = os.path.join(os.getcwd(),"oxasl_display", subject_name)
    #ori_subject = os.path.join(subject_path, "oxasl_distcorr_voxel_calib", "perfusion_calib.nii.gz")
    pwi_image_descriptions = ["PWI without distortion correction and motion correction", "PWI without distortion correction but with motion correction", "PWI with distortion correction and motion correction"]
    pwi_names = [os.path.join('preproc', "pwi.nii.gz"), os.path.join('moco', "pwi.nii.gz"), os.path.join('corrected', "pwi.nii.gz")]
    
    new_subject = os.path.join(subject_path, "oxasl_results", pwi_names[pwi_index] ) 
    pwi_cor_subject = os.path.join(subject_path, "oxasl_results", pwi_names[pwi_index2] )
    
    try:
        new_img = nib.load(new_subject)
        new_img_arr=new_img.get_fdata()
        new_img_arr=np.squeeze(new_img_arr)        
        
        new_img2 = nib.load(pwi_cor_subject)
        new_img_arr2=new_img2.get_fdata()
        new_img_arr2=np.squeeze(new_img_arr2)
        
        plt.figure(figsize=(10,4))
        plt.subplot(1, 2, 1)
        #new_image = getMasked(mask_img_arr, new_img_arr)
        new_img_arr = new_img.dataobj[:,:,slice_index]
        #new_image_arr[new_image_arr == 0] = np.nan
        new_img_arr=np.rot90(new_img_arr)
        plt.imshow(new_img_arr, cmap='gray')
        #plt.set_cmap('RdGy')
        plt.colorbar()
        plt.title(pwi_image_descriptions[pwi_index])
        plt.show()
        
        
        plt.subplot(1, 2, 2)
        #new_image = getMasked(mask_img_arr, new_img_arr)
        new_img_arr2 = new_img2.dataobj[:,:,slice_index]
        #new_image_arr[new_image_arr == 0] = np.nan
        new_img_arr2=np.rot90(new_img_arr2)
        plt.imshow(new_img_arr2, cmap='gray')
        #plt.set_cmap('RdGy')
        plt.colorbar()
        plt.title(pwi_image_descriptions[pwi_index2])
        plt.show()
        
        
    except Exception as e:
        print(str(e))
        #print("File doesn't exist.")

        
_ = interact(show_difference_map,
             subject_name=Subject_dropdown, 
             pwi_index=PWI_dropdown,
             pwi_index2=PWI_dropdown2,
            slice_index=slice_slider)

interactive(children=(Dropdown(description='Subject:', options=('sub-1', 'sub-2', 'sub-3', 'sub-4', 'sub-5', '…

# Perfusion&perfusion_calib 

In [6]:

Image_options = [("Perfusion", 0),('Perfusion_calib', 1)]
Image_dropdown = ipywidgets.Dropdown(options=Image_options,
                                value=1,
                                description="Image Type:")

PVC_options = [("No", 0),('Yes', 1)]
PVC_dropdown = ipywidgets.Dropdown(options=PVC_options,
                                value=0,
                                description="PVC:")


Subject_options = subjects_list
Subject_dropdown = ipywidgets.Dropdown(options=Subject_options,
                                value='sub-1',
                                description="Subject:")

slice_slider = ipywidgets.IntSlider(value=6,
                                    min=0,
                                    max=12,
                                    step=1,
                                    description='Slice:',)

calib_list = [('voxelwise(default)', 0), ("csf", 1), ("wm", 2)]
Calib_options = calib_list
Calib_dropdown = ipywidgets.Dropdown(options=Calib_options,
                                value=0,
                                description="Calibration method:")


def show_subject(subject_name, image_index, pvc_index, slice_index, calib_index):
    subject_id = int(subject_name[4:])
    subject_path = os.path.join(os.getcwd(),"oxasl_display", subject_name)
    native_space_nonpvc_dir = os.path.join(subject_path, "oxasl_results", 'output', 'native')
    native_space_pvc_dir = os.path.join(subject_path, "oxasl_results", 'output_pvcorr', 'native')
    calib_folder = ['calib_voxelwise', 'calib_refregion_csf','calib_refregion_wm']
    mask_subject = os.path.join(native_space_pvc_dir, 'mask.nii.gz')
    
    perfusion_path_list= []
    #mask_list = []
    perfusion_calib_path_list = []
    for calib_f in calib_folder:
        if pvc_index:
            perfusion_path_list.append(os.path.join(native_space_pvc_dir, 'perfusion.nii.gz'))
            #mask_list.append(os.path.join(native_space_pvc_dir, 'mask.nii.gz'))
            perfusion_calib_path_list.append(os.path.join(native_space_pvc_dir, calib_f,'perfusion.nii.gz'))
        else:
            perfusion_path_list.append(os.path.join(native_space_nonpvc_dir, 'perfusion.nii.gz'))
            #mask_list.append(os.path.join(native_space_nonpvc_dir, 'mask.nii.gz'))
            perfusion_calib_path_list.append(os.path.join(native_space_nonpvc_dir, calib_f, 'perfusion.nii.gz'))
    
    image_path =[perfusion_path_list, perfusion_calib_path_list]
    image_desc = ['Perfusion', 'Perfusion_calib']
    try:
        
        ori_img = nib.load(image_path[image_index][calib_index])
        mask_img = nib.load(mask_subject)
        ori_img_arr=ori_img.get_fdata()
        ori_img_arr=np.squeeze(ori_img_arr)
        mask_img_arr=mask_img.get_fdata()
        mask_img_arr=np.squeeze(mask_img_arr)
        
        #ori_image = nib.Nifti1Image(ori_img_arr, affine=np.eye(4))
        
        masked_ori_image = getMasked(mask_img_arr, ori_img_arr)
        ori_image = nib.Nifti1Image(ori_img_arr, affine=np.eye(4))
        ori_image_arr = ori_image.dataobj[:,:,slice_index]
        
        plt.figure(figsize=(10,4))
        ax = plt.subplot(1, 2, 1)
        ori_image_arr=np.rot90(ori_image_arr)
        plt.imshow(ori_image_arr, cmap='gray')
        plt.title(image_desc[image_index])
        #plt.set_cmap('RdGy')
        plt.colorbar()
        plt.show()
        
        plt.subplot(1, 2, 2)
        #plt.hist(ori_image_arr.flatten()) 
        n, bins, patches = plt.hist(x=masked_ori_image.dataobj.flatten(), bins='auto', alpha=0.7, rwidth=0.8, color='#0504aa')
        plt.grid(axis='y', alpha=0.75)
        plt.title("Histgram of whole volume")
        plt.xlabel('Perfusion value')
        plt.ylabel('Number')
        plt.show()
        
    except Exception as e:
        print(str(e))
        #print("Region_analysis file doesn't exist.")

_ = interact(show_subject,
             subject_name=Subject_dropdown, 
             image_index=Image_dropdown,
             pvc_index=PVC_dropdown, 
             slice_index=slice_slider,
             calib_index=Calib_dropdown
             )

interactive(children=(Dropdown(description='Subject:', options=('sub-1', 'sub-2', 'sub-3', 'sub-4', 'sub-5', '…

# Motion Estimates

In [7]:
Subject_options = subjects_list
Subject_dropdown = ipywidgets.Dropdown(options=Subject_options,
                                value='sub-1',
                                description="Subject:")

def show_difference_map(subject_name):
    subject_id = int(subject_name[4:])
    processed_path = os.path.join(os.getcwd(),'oxasl_display', subject_name, 'processed')
    rots_path = os.path.join(processed_path, subject_name+'_rots.png')
    trans_path = os.path.join(processed_path, subject_name+'_trans.png')
    
    try:
       
        plt.figure(figsize=(10,8))
        plt.subplot(2, 1, 1)
        img_rot = Image.open(rots_path)
        plt.imshow(img_rot)
        plt.title(subject_name+" Rotation series") 
        plt.axis('off')
        plt.show()

        plt.subplot(2, 1, 2)
        img_tran = Image.open(trans_path)
        plt.title(subject_name+" Translations series") 
        plt.imshow(img_tran)
        plt.axis('off')
        plt.show()
    except Exception as e:
        print(str(e))
        print("Region_analysis file doesn't exist.")

_ = interact(show_difference_map,
             subject_name=Subject_dropdown)

interactive(children=(Dropdown(description='Subject:', options=('sub-1', 'sub-2', 'sub-3', 'sub-4', 'sub-5', '…